In [26]:
#!/usr/bin/env python

from fanuc_pcdk_client import FanucPCDKClient
from robot_loc_world import RobotLocWorld
from robot_loc_joint import RobotLocJoint
from time import sleep
from copy import deepcopy
import time

ROBOT_SPEED = 30
ROBOT_SPEED_LINEAR = 70

In [2]:
# Robot init
robot = FanucPCDKClient()
robot.connect()

# Run teach pendant program.
robot.run_program('PCDK')
robot.reset_alarms()

# Adjust the speed of the robot
robot.set_speed(ROBOT_SPEED)

Connecting to robot at 192.168.1.1
Robot Status: Connected



In [12]:
# USE THESE TO GET WORLD POSITIONS AND JOINT POSITIONS

# Retrieve the current world XYZWPR coordinates of the robot
loc_world = robot.get_position("world")
print(f'Current world location: {loc_world}\n')

# Retrieve the current joint coordinates of the robot
loc_joint = robot.get_position("joint")
print(f'Current joint location: {loc_joint}\n')

Current world location: x=399.492; y=2.950; z=-161.394; w=179.950; p=-5.962; r=-1.608; f=False; u=True; t=True

Current joint location: j1=0.590; j2=28.926; j3=-35.070; j4=0.369; j5=-48.975; j6=-2.429



====== LEFT
Current world location: x=22.353, y=-407.048, z=-216.317, w=179.841, p=-2.785, r=-88.876, f=False, u=True, t=True

Current joint location: j1=-86.701, j2=39.541, j3=-41.481, j4=0.369, j5=-45.743, j6=-2.430


Current world location: x=22.350, y=-407.028, z=-220.447, w=179.826, p=-2.383, r=-88.875, f=False, u=True, t=True

Current joint location: j1=-86.701, j2=40.379, j3=-41.883, j4=0.369, j5=-45.743, j6=-2.429

====== RIGHT
Current world location: x=6.208, y=399.484, z=-218.712, w=179.713, p=0.205, r=87.090, f=False, u=True, t=True

Current joint location: j1=89.277, j2=40.596, j3=-41.241, j4=0.369, j5=-48.975, j6=-2.429


In [23]:

# IMPORTANT DATA THAT NEEDS TO BE INPUTTED
# EXPERIMENT A LITTLE BIT, USING JOINT POSITIONS MIGHT MAKE THIS ONE EASIER NOT SURE

VERTICAL_SPACING = 26 # vertical center to center spacing between pegs, change accordingly

### NEEDS TO BE CALIBRATED FOR BOTTOM MOST PEG

left_peg_pose = RobotLocWorld(x=22.350, y=-407.028, z=-220.447, w=179.826, p=-2.383, r=-88.875, f=False, u=True, t=True)

left_peg_joints = RobotLocJoint(j1=-86.701, j2=40.379, j3=-41.883, j4=0.369, j5=-45.743, j6=-2.429)

right_peg_pose = RobotLocWorld(x=6.208, y=399.484, z=-218.712, w=179.713, p=0.205, r=87.090, f=False, u=True, t=True)

right_peg_joints = RobotLocJoint(j1=89.277, j2=40.596, j3=-41.241, j4=0.369, j5=-48.975, j6=-2.429)

home_pose = RobotLocWorld(x=407.028, y=2.950, z=-161.394, w=179.950, p=-5.962, r=-1.608, f=False, u=True, t=True)

In [20]:
def generate_trajectories(num_pegs):
    trajectory = []
    
    trajectory.append(home_pose)
    
    left_above = deepcopy(left_peg_pose)
    left_above.z += 5*VERTICAL_SPACING
    
    right_above = deepcopy(right_peg_pose)
    right_above.z += 5*VERTICAL_SPACING
    
    
    for i in reversed(range(1, num_pegs+1)):
        trajectory.append(left_above)
        left = deepcopy(left_peg_pose)
        right = deepcopy(right_peg_pose)
        left.z += (i-1)*VERTICAL_SPACING
        trajectory.append(left)
        trajectory.append('close') # may need to be swapped if on LR Mate 200iD
        trajectory.append(left_above)
        trajectory.append(home_pose)
        trajectory.append(right_above)
        right.z += (3-i)*VERTICAL_SPACING
        trajectory.append(right)
        trajectory.append('open') # may need to be swapped if on LR Mate 200iD
        trajectory.append(right_above)
        trajectory.append(home_pose)
    
    return trajectory

In [25]:
robot.close_gripper()

In [24]:
requested_pegs = 0
success = 0
def execute_stacking(pegs):
    print("\n\n===============    STARTING PEG STACKING    ===============")
    success = 0
    requested_pegs = pegs
    trajectory = generate_trajectories(pegs)
        
    for pose in trajectory:
        pegs_moved = success
        time.sleep(0.5)
        if(pose == 'close'):
            robot.open_gripper()
            print("Picking up the peg #" + str(pegs_moved + 1) + " now!")
        elif(pose == 'open'):
            robot.close_gripper()
            success += 1
            print(" peg #" + str(pegs_moved + 1) + " has been completed!")
        else:
            robot.move(pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)
    
    failed = requested_pegs - success
    
    print("\n\n============================\n      STACKING SUMMARY\n============================")
    print("Successfully stacked " + str((success)) + " pegs")
    print("Failed to stack " + str((failed)) + " pegs")

In [29]:
terminate = False
user_input = 0
started = False
while not terminate:
    try:
        if(not started):
            print("===============  WELCOME TO PEG STACKING  ===============")
            start = True
            
        user_input = (input('How many pegs are there to move? (press q to quit)\n> '))
        user_input = (user_input) if user_input == 'q' else int(user_input)
        if(user_input == 'q'):
            print("===============    TERMINATING PEG STACKING    ===============")
            terminate = True
            break
        elif(user_input > 0):
            
            '''
            ACTUAL PEG STACK STUFF GOES HERE
            '''
            execute_stacking(user_input)
            
        else:
            print("Invalid choice, please try again.")
            terminate = True
        
        if(user_input < 1):
            terminate = True
            
    except ValueError:
        print("Not a valid number, please try again")

===============  WELCOME TO PEG STACKING  ===============
How many pegs are there to move? (press q to quit)
> q
===============    TERMINATING PEG STACKING    ===============
